In [16]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from dask.diagnostics import ProgressBar
from geocube.api.core import make_geocube

PATH = "D:\World Bank\CLIENT v2"
DATA_RAW = rf"{PATH}\Data\Data_raw"
DATA_PROC = rf"{PATH}\Data\Data_proc"
DATA_OUT = rf"{PATH}\Data\Data_out"

In [17]:
gpd.__version__

'0.14.4'

In [9]:
data = ["D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat4_1951_0.25.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat3_1951_0.25.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat2_1951_0.25.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat1_1951_0.25.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat5_1951_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat4_1951_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat3_1951_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat2_1951_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat1_1951_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat5_1951_1.0.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat4_1951_1.0.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat3_1951_1.0.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat2_1951_1.0.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat1_1951_1.0.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat5_1951_0.5.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat4_1951_0.5.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat3_1951_0.5.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat2_1951_0.5.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat1_1951_0.5.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat5_1951_0.25.nc"]

data2 = [
# "D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat2_1952_0.25.nc"
# "D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat1_1952_0.25.nc"
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat5_1952_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat4_1952_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat3_1952_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat2_1952_0.1.nc",
"D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat1_1952_0.1.nc",
# "D:\World Bank\CLIENT v2\Data\Data_proc\hurricanes\IBTrACS_cat3_1952_0.25.nc"
]
import xarray as xr
xr.open_mfdataset([data, data2], combine='nested', concat_dim="year")

ValueError: The supplied objects do not form a hypercube because sub-lists do not have consistent lengths along dimension 0

In [85]:
gdf = gpd.read_file(r"D:\Datasets\International Best Track Archive for Climate Stewardship (IBTrACS)\IBTrACS.ALL.list.v04r01.lines.shp")#, "BASIN", "SUBBASIN", "NAME", "ISO_TIME", "LAT", "LON", "WMO_WIND", "WMO_PRES", "WMO_AGENCY", "TRACK_TYPE"])
gdf["WMO_WIND"] = gdf["WMO_WIND"].ffill()
gdf[gdf.SID == "2021323S10103"].explore(column="WMO_WIND")